### Exponential Thermal Tensor Network Renormalization Group (XTRG)

Date: <span style = "color: wheat"> 23.07.2025 </span> | Author of the code: <span style = "color: wheat"> Matthias Pawlik, Damiano Aliverti </span>

The Exponential Tensor Renormalization Group (XTRG) is a powerful numerical method for computing the thermal density matrix $\hat{\rho} = e^{− \beta \hat{H}}$ of finite-size quantum systems, where $\beta$ is the inverse temperature and $\hat{H}$ is the many-body Hamiltonian. In this problem, you will implement the XTRG algorithm and use it to compute the partition function of a one-dimensional ($1\text{D}$) XY model from high temperatures ($\beta \sim 10^{−6}$) down to low temperatures ($\beta \sim 1$).

<span style = "color: wheat"> [1]  </span> B.-B. Chen, L. Chen, Z. Chen, W. Li, and A. Weichselbaum, Phys. Rev. X 8, 031082 (2018), https://doi.org/10.1103/PhysRevX.8.031082.

<span style = "color: wheat"> (a) </span> Initialize the thermal density matrix as a matrix product operator (MPO) using linear initialization, $\rho(\beta_0) \approx \mathbb{I} − \beta_0 H$, as described in Appendix C.2 of Ref. [1]. Use $\beta_0 = 10^{−6}$ as the initial inverse temperature.

We define the Hamiltonian of the spin-$\frac{1}{2}$ XY-chain of length $\mathcal{L}$ with open boundary conditions (OPC) to $\hat{H} = J \sum_{l = 1}^{\mathcal{L}-1} (\hat{S}^x_l \hat{S}^x_{l+1} + \hat{S}^y_l \hat{S}^y_{l+1}) \equiv J \sum_{l = 1}^{\mathcal{L}-1} (\hat{S}^+_l \hat{S}^-_{l+1} + \hat{S}^-_l \hat{S}^+_{l+1})$.

<span style = "color: SkyBlue"> Solution: </span>

In [ ]:
include("source/MPO.jl")
using .MPO

In [ ]:
function linear_initialization(L::Int; J::Float64=1.0, beta0::Float64=1e-6)

    # Generate the MPO Hamiltonian for the XY-Model
    H_mpo = xychain_mpo(L, J)

    # Construct an MPO representation of the identity operator
    Id_mpo = identity_mpo(L)

    # Multiply H with inverse initial temperature
    H_mpo = [(beta0)^(-1/L) * W for W in H_mpo]
    
    # Add the MPOs and obtain new local tensors with bond dimension D = 4 + 1
    rho0 = add_mpo(H_mpo, Id_mpo)

    return rho0

end

linear_initialization (generic function with 1 method)

<span style = "color: wheat"> (b) </span> Implement the XTRG algorithm following the strategy in Sec. II of Ref.[1]. The key idea is to
iteratively double the inverse temperature, $\rho(2 \beta) = \rho(\beta) \times \rho(\beta)$, by contracting the MPO with itself. After each multiplication, the MPO bond dimension increases and must be truncated. This can be done variationally using DMRG-type sweeping, as detailed in Appendix D of Ref. [1].

<span style = "color: SkyBlue"> Solution: </span>

In [ ]:
include("source/XTRG.jl")
using .XTRG

<span style = "color: wheat"> (c) </span> Apply your XTRG implementation to the 1D XY-model of length $L = 10$. Perform $20$ XTRG
steps starting from $\beta_0 = 10^{−6}$, so that the final inverse temperature is $\beta = 2^{20} \beta_0$. At each step, compute the partition function $Z = \text{Tr}(\rho(\beta))$. Compare your numerical results with the analytical solution provided in Appendix F of Ref. [1] over the full temperature range.

<span style = "color: SkyBlue"> Solution: </span>

In [ ]:
# Parameter setting
L = 10
beta0 = 1e-6
Nsteps = 20

# Linear initialization 
rho0 = linear_initialization(L);

# Execute the XTRG Algorithm
betas, Zs, rhos = XTRG_algorithm(beta0, Nsteps, rho0)

# # # Started Variational Optimization # # #Temperature update: 1.0e-6 ---> 2.0e-6# of sites = 10 | square mode = true | # of sweeps = 5 x 2
 | convergence = 1.0e-10
| Nkeep = 6 (Dmax = 100, alpha = 1.1) | tolerance = 1.0e-12

LoadError: MethodError: no method matching *(::Array{ComplexF64, 4}, ::LinearAlgebra.Diagonal{Float64, Vector{Float64}})
The function `*` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m)
[0m[90m   @[39m [90mBase[39m [90m[4moperators.jl:596[24m[39m
[0m  *([91m::LinearAlgebra.Transpose{T, <:AbstractVector} where T[39m, ::LinearAlgebra.Diagonal, [91m::AbstractVector[39m)
[0m[90m   @[39m [36mLinearAlgebra[39m [90mC:\Users\MPawl\.julia\juliaup\julia-1.11.6+0.x64.w64.mingw32\share\julia\stdlib\v1.11\LinearAlgebra\src\[39m[90m[4mdiagonal.jl:953[24m[39m
[0m  *([91m::LinearAlgebra.Transpose{T, <:AbstractVector} where T[39m, ::AbstractMatrix)
[0m[90m   @[39m [36mLinearAlgebra[39m [90mC:\Users\MPawl\.julia\juliaup\julia-1.11.6+0.x64.w64.mingw32\share\julia\stdlib\v1.11\LinearAlgebra\src\[39m[90m[4mmatmul.jl:96[24m[39m
[0m  ...


In [13]:
typeof(beta0)

Float64

In [ ]:
# Analytical solution for the partition function of the XY-Model with OBC

function partition_function_analytical(beta, L, J=1.0)
    Z = 1.0
    for k in 1:L
        epsilon_k = J * cos(k * pi / (L + 1))
        Z *= (1 + exp(-beta * epsilon_k))
    end
    return Z 
end

In [ ]:
# Compute analytical values for simulated inverse temperatures
Zs_analytical = [partition_function_analytical(beta, L) for beta in betas]

# Plot partition function against inverse temperature
plot(betas, Zs, 
     label="XTRG Numerical", 
     marker=:circle, 
     markersize=4,
     linewidth=2,
     xscale=:log10,
     yscale=:log10)

plot!(betas, Zs_analytical, 
      label="Analytical", 
      linestyle=:dash,
      linewidth=2)

xlabel!("Inverse temperature \\beta")
ylabel!("Partition Function Z(\\beta)")
title!("XY Model Partition Function: XTRG vs Analytical Solution")
legend!(:bottomleft)
grid!(true)

# Display the plot
display(current())